# CLASH OF NEIBORHOOD

## IBM DATA SCIENCE CAPSTONE PROJECT

### WEEK 3 PART 3

#### Headings are made in the form of comment on top of every cell

In [1]:
#importing libraries

import pandas as pd
import numpy as np
from geopy.geocoders import Nominatim #to convert address into latitude and longitude
import requests
from bs4 import BeautifulSoup
import folium
import json
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as color

In [2]:
# coping week 3 part 1

data=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(data,'html.parser')
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned': #removing null values
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df_group = df.groupby(["PostalCode", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_group.head(20)

PostalCode      Borough                                       Neighborhood
0         M1B  Scarborough                                     Malvern, Rouge
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek
2         M1E  Scarborough                  Guildwood, Morningside, West Hill
3         M1G  Scarborough                                             Woburn
4         M1H  Scarborough                                          Cedarbrae
5         M1J  Scarborough                                Scarborough Village
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West
9         M1N  Scarborough                        Birch Cliff, Cliffside West
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...
11        M1R  Scarborough                                  Wexford, Maryvale
12        M1S  Scarborough                                          Agincourt
13        M1T  Scarborough            Clarks Corners, Tam O'Shanter, Sullivan
14        M1V  Scarborough  Milliken, Agincourt North, Steeles East, L'Amo...
15        M1W  Scarborough                      Steeles West, L'Amoreaux West
16        M1X  Scarborough                                        Upper Rouge
17        M2H   North York                                  Hillcrest Village
18        M2J   North York                       Fairview, Henry Farm, Oriole
19        M2K   North York                                    Bayview Village

In [3]:
#coping week 3 part 2

cords=pd.read_csv("Geospatial_Coordinates.csv")
cords.rename(columns={"Postal Code": "PostalCode"},inplace=True)
df_new = df_group.merge(cords, on="PostalCode", how="left")
df_new.head(20)

PostalCode      Borough                                       Neighborhood  \
0         M1B  Scarborough                                     Malvern, Rouge   
1         M1C  Scarborough             Rouge Hill, Port Union, Highland Creek   
2         M1E  Scarborough                  Guildwood, Morningside, West Hill   
3         M1G  Scarborough                                             Woburn   
4         M1H  Scarborough                                          Cedarbrae   
5         M1J  Scarborough                                Scarborough Village   
6         M1K  Scarborough        Kennedy Park, Ionview, East Birchmount Park   
7         M1L  Scarborough                    Golden Mile, Clairlea, Oakridge   
8         M1M  Scarborough    Cliffside, Cliffcrest, Scarborough Village West   
9         M1N  Scarborough                        Birch Cliff, Cliffside West   
10        M1P  Scarborough  Dorset Park, Wexford Heights, Scarborough Town...   
11        M1R  Scarborough                                  Wexford, Maryvale   
12        M1S  Scarborough                                          Agincourt   
13        M1T  Scarborough            Clarks Corners, Tam O'Shanter, Sullivan   
14        M1V  Scarborough  Milliken, Agincourt North, Steeles East, L'Amo...   
15        M1W  Scarborough                      Steeles West, L'Amoreaux West   
16        M1X  Scarborough                                        Upper Rouge   
17        M2H   North York                                  Hillcrest Village   
18        M2J   North York                       Fairview, Henry Farm, Oriole   
19        M2K   North York                                    Bayview Village   

     Latitude  Longitude  
0   43.806686 -79.194353  
1   43.784535 -79.160497  
2   43.763573 -79.188711  
3   43.770992 -79.216917  
4   43.773136 -79.239476  
5   43.744734 -79.239476  
6   43.727929 -79.262029  
7   43.711112 -79.284577  
8   43.716316 -79.239476  
9   43.692657 -79.264848  
10  43.757410 -79.273304  
11  43.750072 -79.295849  
12  43.794200 -79.262029  
13  43.781638 -79.304302  
14  43.815252 -79.284577  
15  43.799525 -79.318389  
16  43.836125 -79.205636  
17  43.803762 -79.363452  
18  43.778517 -79.346556  
19  43.786947 -79.385975

In [4]:
#accessing latitude and longitude values of Toronto

name="Toronto"
geoloc = Nominatim(user_agent="my_application")
loc = geoloc.geocode(name)
lat = loc.latitude
lon = loc.longitude
print("Toronto {} {} ".format(lat,lon))#will be required to generate map of toronto

Toronto 43.6534817 -79.3839347 


In [5]:
#creating map using folium

map = folium.Map(location=[lat,lon], zoom_start=10)
for lt,lg,b,n in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{},{}'.format(n,b)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lt,lg], radius=5, popup=label, color='blue', fill=True, fill_color='red', fill_opacity=0.7).add_to(map)

map

In [6]:
#working with boroughs that only contains the word TORONTO

names=list(df_new.Borough.unique())
names_with_toronto=[]
for name in names:
    if "toronto" in name.lower():
        names_with_toronto.append(name)
        
names_with_toronto

['East Toronto',
 'East York/East Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto Stn A',
 'West Toronto',
 'East Toronto Business']

In [7]:
#creating a new dataframe with toronto

df_new=df_new[df_new['Borough'].isin(names_with_toronto)].reset_index(drop=True)
df_new.head(20)

PostalCode                 Borough  \
0         M4E            East Toronto   
1         M4J  East York/East Toronto   
2         M4K            East Toronto   
3         M4L            East Toronto   
4         M4M            East Toronto   
5         M4N         Central Toronto   
6         M4P         Central Toronto   
7         M4R         Central Toronto   
8         M4S         Central Toronto   
9         M4T         Central Toronto   
10        M4V         Central Toronto   
11        M4W        Downtown Toronto   
12        M4X        Downtown Toronto   
13        M4Y        Downtown Toronto   
14        M5A        Downtown Toronto   
15        M5B        Downtown Toronto   
16        M5C        Downtown Toronto   
17        M5E        Downtown Toronto   
18        M5G        Downtown Toronto   
19        M5H        Downtown Toronto   

                                         Neighborhood   Latitude  Longitude  
0                                         The Beaches  43.676357 -79.293031  
1                                  The Danforth  East  43.685347 -79.338106  
2                        The Danforth West, Riverdale  43.679557 -79.352188  
3                      India Bazaar, The Beaches West  43.668999 -79.315572  
4                                     Studio District  43.659526 -79.340923  
5                                       Lawrence Park  43.728020 -79.388790  
6                                    Davisville North  43.712751 -79.390197  
7                                  North Toronto West  43.715383 -79.405678  
8                                          Davisville  43.704324 -79.388790  
9                         Moore Park, Summerhill East  43.689574 -79.383160  
10  Summerhill West, Rathnelly, South Hill, Forest...  43.686412 -79.400049  
11                                           Rosedale  43.679563 -79.377529  
12                        St. James Town, Cabbagetown  43.667967 -79.367675  
13                               Church and Wellesley  43.665860 -79.383160  
14                          Regent Park, Harbourfront  43.654260 -79.360636  
15                           Garden District, Ryerson  43.657162 -79.378937  
16                                     St. James Town  43.651494 -79.375418  
17                                        Berczy Park  43.644771 -79.373306  
18                                 Central Bay Street  43.657952 -79.387383  
19                           Richmond, Adelaide, King  43.650571 -79.384568

In [8]:
df_new.shape

(39, 5)

In [9]:
map_toronto = folium.Map(location=[lat,lon], zoom_start=10)
for lt,lg,b,n in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{},{}'.format(n,b)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lt,lg], radius=5, popup=label, color='blue', fill=True, fill_color='red', fill_opacity=0.7).add_to(map_toronto)

map_toronto

In [10]:
#using FOURSQUARE API to explore neighborhood

#my credentials
id="PT0LDSCGFFXT4HWLLN2UJB3M4GBMGOAJHWSQRTEE4G2SAA3E"
secret="Z1HE1SXNLPP4KOKYTTLW3DOUGCS3SNEDKCXMDJQDBDUDJBKP"
version="20180605"

radius=500
limit=100
venues=[]
for lt,lg, post, b, n in zip(df_new['Latitude'], df_new['Longitude'], df_new['PostalCode'], df_new['Borough'], df_new['Neighborhood']):
    url="https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(id,secret,version,lt,lg,radius,limit)
    results=requests.get(url).json()["response"]["groups"][0]["items"]
    for venue in results:
        venues.append((post,b,n,lt,lg,venue['venue']['name'],venue['venue']['location']['lat'],venue['venue']['location']['lng'],venue['venue']['categories'][0]['name']))
    
#venues list into dataframe

df_venues=pd.DataFrame(venues)
df_venues.columns=['PostalCode', 'Borough', 'Neighborhood', 'BoroughLat', 'BoroughLong', 'VenueName','VenueLat', 'VenueLong', 'VenueCategory']
print(df_venues.shape)
df_venues.head(20)

(1583, 9)


PostalCode                 Borough                  Neighborhood  \
0         M4E            East Toronto                   The Beaches   
1         M4E            East Toronto                   The Beaches   
2         M4E            East Toronto                   The Beaches   
3         M4E            East Toronto                   The Beaches   
4         M4J  East York/East Toronto            The Danforth  East   
5         M4J  East York/East Toronto            The Danforth  East   
6         M4J  East York/East Toronto            The Danforth  East   
7         M4K            East Toronto  The Danforth West, Riverdale   
8         M4K            East Toronto  The Danforth West, Riverdale   
9         M4K            East Toronto  The Danforth West, Riverdale   
10        M4K            East Toronto  The Danforth West, Riverdale   
11        M4K            East Toronto  The Danforth West, Riverdale   
12        M4K            East Toronto  The Danforth West, Riverdale   
13        M4K            East Toronto  The Danforth West, Riverdale   
14        M4K            East Toronto  The Danforth West, Riverdale   
15        M4K            East Toronto  The Danforth West, Riverdale   
16        M4K            East Toronto  The Danforth West, Riverdale   
17        M4K            East Toronto  The Danforth West, Riverdale   
18        M4K            East Toronto  The Danforth West, Riverdale   
19        M4K            East Toronto  The Danforth West, Riverdale   

    BoroughLat  BoroughLong                           VenueName   VenueLat  \
0    43.676357   -79.293031                   Glen Manor Ravine  43.676821   
1    43.676357   -79.293031  The Big Carrot Natural Food Market  43.678879   
2    43.676357   -79.293031                 Grover Pub and Grub  43.679181   
3    43.676357   -79.293031                       Upper Beaches  43.680563   
4    43.685347   -79.338106                        Aldwych Park  43.684901   
5    43.685347   -79.338106                            The Path  43.683923   
6    43.685347   -79.338106                  Sammon Convenience  43.686951   
7    43.679557   -79.352188                       MenEssentials  43.677820   
8    43.679557   -79.352188                            Pantheon  43.677621   
9    43.679557   -79.352188                        Dolce Gelato  43.677773   
10   43.679557   -79.352188                     Cafe Fiorentina  43.677743   
11   43.679557   -79.352188                          La Diperie  43.677702   
12   43.679557   -79.352188                Moksha Yoga Danforth  43.677622   
13   43.679557   -79.352188                               Mezes  43.677962   
14   43.679557   -79.352188                 Valley Farm Produce  43.677999   
15   43.679557   -79.352188              Louis Cifer Brew Works  43.677663   
16   43.679557   -79.352188             Messini Authentic Gyros  43.677704   
17   43.679557   -79.352188                          Alexandros  43.678304   
18   43.679557   -79.352188                           7 Numbers  43.677062   
19   43.679557   -79.352188                   The Auld Spot Pub  43.677335   

    VenueLong            VenueCategory  
0  -79.293942                    Trail  
1  -79.297734        Health Food Store  
2  -79.297215                      Pub  
3  -79.292869             Neighborhood  
4  -79.341091                     Park  
5  -79.335007                     Park  
6  -79.335007        Convenience Store  
7  -79.351265           Cosmetics Shop  
8  -79.351434         Greek Restaurant  
9  -79.351187           Ice Cream Shop  
10 -79.350115       Italian Restaurant  
11 -79.352265           Ice Cream Shop  
12 -79.352116              Yoga Studio  
13 -79.350196         Greek Restaurant  
14 -79.349969  Fruit & Vegetable Store  
15 -79.351313                  Brewery  
16 -79.350480         Greek Restaurant  
17 -79.349486         Greek Restaurant  
18 -79.353934       Italian Restaurant  
19 -79.353130                      Pub

In [11]:
print("There are {} categories".format(len(df_venues['VenueCategory'].unique())))

There are 234 categories


In [12]:
df_venues["VenueCategory"].unique()[:50]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Park',
       'Convenience Store', 'Cosmetics Shop', 'Greek Restaurant',
       'Ice Cream Shop', 'Italian Restaurant', 'Yoga Studio',
       'Fruit & Vegetable Store', 'Brewery', 'Restaurant', 'Dessert Shop',
       'Juice Bar', 'Pizza Place', 'Bookstore', 'Furniture / Home Store',
       'Bubble Tea Shop', 'Spa', 'Grocery Store', 'Coffee Shop', 'Bakery',
       'Tibetan Restaurant', 'Indian Restaurant', 'Caribbean Restaurant',
       'Café', 'Lounge', 'Frozen Yogurt Shop', 'Liquor Store',
       'American Restaurant', 'Gym', 'Fish & Chips Shop',
       'Fast Food Restaurant', 'Sushi Restaurant', 'Pet Store',
       'Steakhouse', 'Movie Theater', 'Sandwich Place', 'Playground',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar', 'Cheese Shop',
       'Seafood Restaurant', 'Thai Restaurant', 'Comfort Food Restaurant',
       'Middle Eastern Restaurant', 'Stationery Store'], dtype=object)

In [13]:
df_venues.rank().head(20)

PostalCode  Borough  Neighborhood  BoroughLat  BoroughLong  VenueName  \
0          2.5   1355.5        1405.5      1439.5       1581.5      505.0   
1          2.5   1355.5        1405.5      1439.5       1581.5     1324.0   
2          2.5   1355.5        1405.5      1439.5       1581.5      526.0   
3          2.5   1355.5        1405.5      1439.5       1581.5     1495.0   
4          6.0   1424.0        1409.0      1488.0       1543.0       30.0   
5          6.0   1424.0        1409.0      1488.0       1543.0     1391.0   
6          6.0   1424.0        1409.0      1488.0       1543.0     1086.0   
7         28.0   1355.5        1431.0      1462.0       1483.0      795.0   
8         28.0   1355.5        1431.0      1462.0       1483.0      907.0   
9         28.0   1355.5        1431.0      1462.0       1483.0      378.0   
10        28.0   1355.5        1431.0      1462.0       1483.0      223.0   
11        28.0   1355.5        1431.0      1462.0       1483.0      710.0   
12        28.0   1355.5        1431.0      1462.0       1483.0      826.0   
13        28.0   1355.5        1431.0      1462.0       1483.0      813.0   
14        28.0   1355.5        1431.0      1462.0       1483.0     1500.0   
15        28.0   1355.5        1431.0      1462.0       1483.0      749.0   
16        28.0   1355.5        1431.0      1462.0       1483.0      809.0   
17        28.0   1355.5        1431.0      1462.0       1483.0       35.0   
18        28.0   1355.5        1431.0      1462.0       1483.0        6.0   
19        28.0   1355.5        1431.0      1462.0       1483.0     1317.0   

    VenueLat  VenueLong  VenueCategory  
0     1445.0     1582.0         1534.0  
1     1481.5     1580.0          864.5  
2     1483.0     1581.0         1261.0  
3     1484.0     1583.0         1120.5  
4     1488.0     1528.0         1163.0  
5     1487.0     1543.5         1163.0  
6     1494.0     1543.5          577.0  
7     1463.0     1483.0          584.5  
8     1454.0     1480.0          806.5  
9     1462.0     1484.0          920.5  
10    1460.0     1487.0          959.5  
11    1458.0     1475.0          920.5  
12    1455.0     1476.0         1578.0  
13    1465.0     1486.0          806.5  
14    1466.0     1488.0          744.0  
15    1456.0     1481.0          214.5  
16    1459.0     1485.0          806.5  
17    1472.0     1489.0          806.5  
18    1446.0     1468.0          959.5  
19    1450.0     1471.0         1261.0

In [14]:
#categorical variables are of two types - nominal(who do not have any numeric relation among themselves like here name of venues)
#                                       - ordinal(who have some numeric relation among themselves like graduate masters phd)
#we cannot encode venues with numerics in order to rank them
#hence we need to use one hot encoding in order to find out the top 10 venues

df_onehot = pd.get_dummies(df_venues[['VenueCategory']],prefix="",prefix_sep="")
df_onehot['PostalCode']=df_venues['PostalCode']
df_onehot['Borough']=df_venues['Borough']
df_onehot['Neighborhood']=df_venues['Neighborhood']
columns=list(df_onehot.columns[-3:])+list(df_onehot.columns[:-3])
df_onehot=df_onehot[columns]
print(df_onehot.shape)
df_onehot.head()

(1583, 236)


Yoga Studio PostalCode                 Borough  Adult Boutique  Airport  \
0            0        M4E            East Toronto               0        0   
1            0        M4E            East Toronto               0        0   
2            0        M4E            East Toronto               0        0   
3            0        M4E            East Toronto               0        0   
4            0        M4J  East York/East Toronto               0        0   

   Airport Food Court  Airport Gate  Airport Lounge  Airport Service  \
0                   0             0               0                0   
1                   0             0               0                0   
2                   0             0               0                0   
3                   0             0               0                0   
4                   0             0               0                0   

   Airport Terminal  ...  Theater  Theme Restaurant  Tibetan Restaurant  \
0                 0  ...        0                 0                   0   
1                 0  ...        0                 0                   0   
2                 0  ...        0                 0                   0   
3                 0  ...        0                 0                   0   
4                 0  ...        0                 0                   0   

   Toy / Game Store  Trail  Train Station  Vegetarian / Vegan Restaurant  \
0                 0      1              0                              0   
1                 0      0              0                              0   
2                 0      0              0                              0   
3                 0      0              0                              0   
4                 0      0              0                              0   

   Video Game Store  Vietnamese Restaurant  Wine Bar  
0                 0                      0         0  
1                 0                      0         0  
2                 0                      0         0  
3                 0                      0         0  
4                 0                      0         0  

[5 rows x 236 columns]

In [15]:
df_onehotgrouped = df_onehot.groupby(["PostalCode","Borough","Neighborhood"]).mean().reset_index()
print(df_onehotgrouped.shape)
df_onehotgrouped.head(20)

(39, 236)


PostalCode                 Borough  \
0         M4E            East Toronto   
1         M4J  East York/East Toronto   
2         M4K            East Toronto   
3         M4L            East Toronto   
4         M4M            East Toronto   
5         M4N         Central Toronto   
6         M4P         Central Toronto   
7         M4R         Central Toronto   
8         M4S         Central Toronto   
9         M4T         Central Toronto   
10        M4V         Central Toronto   
11        M4W        Downtown Toronto   
12        M4X        Downtown Toronto   
13        M4Y        Downtown Toronto   
14        M5A        Downtown Toronto   
15        M5B        Downtown Toronto   
16        M5C        Downtown Toronto   
17        M5E        Downtown Toronto   
18        M5G        Downtown Toronto   
19        M5H        Downtown Toronto   

                                         Neighborhood  Yoga Studio  \
0                                         The Beaches     0.000000   
1                                  The Danforth  East     0.000000   
2                        The Danforth West, Riverdale     0.024390   
3                      India Bazaar, The Beaches West     0.000000   
4                                     Studio District     0.026316   
5                                       Lawrence Park     0.000000   
6                                    Davisville North     0.000000   
7                                  North Toronto West     0.050000   
8                                          Davisville     0.000000   
9                         Moore Park, Summerhill East     0.000000   
10  Summerhill West, Rathnelly, South Hill, Forest...     0.000000   
11                                           Rosedale     0.000000   
12                        St. James Town, Cabbagetown     0.000000   
13                               Church and Wellesley     0.026316   
14                          Regent Park, Harbourfront     0.023256   
15                           Garden District, Ryerson     0.000000   
16                                     St. James Town     0.000000   
17                                        Berczy Park     0.000000   
18                                 Central Bay Street     0.016129   
19                           Richmond, Adelaide, King     0.000000   

    Adult Boutique  Airport  Airport Food Court  Airport Gate  Airport Lounge  \
0         0.000000      0.0                 0.0           0.0             0.0   
1         0.000000      0.0                 0.0           0.0             0.0   
2         0.000000      0.0                 0.0           0.0             0.0   
3         0.000000      0.0                 0.0           0.0             0.0   
4         0.000000      0.0                 0.0           0.0             0.0   
5         0.000000      0.0                 0.0           0.0             0.0   
6         0.000000      0.0                 0.0           0.0             0.0   
7         0.000000      0.0                 0.0           0.0             0.0   
8         0.000000      0.0                 0.0           0.0             0.0   
9         0.000000      0.0                 0.0           0.0             0.0   
10        0.000000      0.0                 0.0           0.0             0.0   
11        0.000000      0.0                 0.0           0.0             0.0   
12        0.000000      0.0                 0.0           0.0             0.0   
13        0.013158      0.0                 0.0           0.0             0.0   
14        0.000000      0.0                 0.0           0.0             0.0   
15        0.000000      0.0                 0.0           0.0             0.0   
16        0.000000      0.0                 0.0           0.0             0.0   
17        0.000000      0.0                 0.0           0.0             0.0   
18        0.000000      0.0                 0.0           0.0             0.0   
19        0.000000      0.0                 0.0   

In [16]:
#finding out top 10 places

top=10;
indicators=['st','nd','rd']
area=['PostalCode','Borough', 'Neighborhood']
freq=[]
for i in np.arange(top):
    try:
        freq.append('{}{} Most Common Venue'.format(i+1,indicators[i]))
    except:
        freq.append('{}th Most Common Venue'.format(i+1))
columns=area+freq
df_topvenues=pd.DataFrame(columns=columns)
df_topvenues['PostalCode']=df_onehotgrouped['PostalCode']
df_topvenues['Borough']=df_onehotgrouped['Borough']
df_topvenues['Neighborhood']=df_onehotgrouped['Neighborhood']
for i in np.arange(df_onehotgrouped.shape[0]):
    row=df_onehotgrouped.iloc[i,:].iloc[3:]
    row_sorted=row.sort_values(ascending=False)
    df_topvenues.iloc[i,3:]=row_sorted.index.values[0:top]
print(df_topvenues.shape)
df_topvenues.head(40)

(39, 13)


PostalCode                 Borough  \
0         M4E            East Toronto   
1         M4J  East York/East Toronto   
2         M4K            East Toronto   
3         M4L            East Toronto   
4         M4M            East Toronto   
5         M4N         Central Toronto   
6         M4P         Central Toronto   
7         M4R         Central Toronto   
8         M4S         Central Toronto   
9         M4T         Central Toronto   
10        M4V         Central Toronto   
11        M4W        Downtown Toronto   
12        M4X        Downtown Toronto   
13        M4Y        Downtown Toronto   
14        M5A        Downtown Toronto   
15        M5B        Downtown Toronto   
16        M5C        Downtown Toronto   
17        M5E        Downtown Toronto   
18        M5G        Downtown Toronto   
19        M5H        Downtown Toronto   
20        M5J        Downtown Toronto   
21        M5K        Downtown Toronto   
22        M5L        Downtown Toronto   
23        M5N         Central Toronto   
24        M5P         Central Toronto   
25        M5R         Central Toronto   
26        M5S        Downtown Toronto   
27        M5T        Downtown Toronto   
28        M5V        Downtown Toronto   
29        M5W  Downtown Toronto Stn A   
30        M5X        Downtown Toronto   
31        M6G        Downtown Toronto   
32        M6H            West Toronto   
33        M6J            West Toronto   
34        M6K            West Toronto   
35        M6P            West Toronto   
36        M6R            West Toronto   
37        M6S            West Toronto   
38        M7Y   East Toronto Business   

                                         Neighborhood 1st Most Common Venue  \
0                                         The Beaches                 Trail   
1                                  The Danforth  East                  Park   
2                        The Danforth West, Riverdale      Greek Restaurant   
3                      India Bazaar, The Beaches West  Fast Food Restaurant   
4                                     Studio District           Coffee Shop   
5                                       Lawrence Park           Swim School   
6                                    Davisville North      Department Store   
7                                  North Toronto West           Coffee Shop   
8                                          Davisville           Pizza Place   
9                         Moore Park, Summerhill East            Restaurant   
10  Summerhill West, Rathnelly, South Hill, Forest...           Coffee Shop   
11                                           Rosedale                  Park   
12                        St. James Town, Cabbagetown            Restaurant   
13                               Church and Wellesley           Coffee Shop   
14                          Regent Park, Harbourfront           Coffee Shop   
15                           Garden District, Ryerson           Coffee Shop   
16                                     St. James Town           Coffee Shop   
17                                        Berczy Park           Coffee Shop   
18                                 Central Bay Street           Coffee Shop   
19                           Richmond, Adelaide, King           Coffee Shop   
20  Harbourfront East, Union Station, Toronto Islands           Coffee Shop   
21           Toronto Dominion Centre, Design Exchange           Coffee Shop   
22                     Commerce Court, Victoria Hotel           Coffee Shop   
23                                           Roselawn          Home Service   
24                           Forest Hill North & West                 Trail   
25                The Annex, North Midtown, Yorkville        Sandwich Place   
26                     University of Toronto, Harbord                  Café   
27          Kensington Market, Chinatown, Grange Park                  Café   
28  CN Tower, King and Spadina, Railway Lands, Har...      Airport Term

In [17]:
#using KMeans to cluster areas in 6 clusters

k=6
df_cluster=df_onehotgrouped.drop(["PostalCode","Borough","Neighborhood"],1)
kmeans=KMeans(n_clusters=k,random_state=0).fit(df_cluster)
kmeans.labels_[0:10]

array([4, 0, 1, 4, 1, 3, 4, 1, 4, 2])

In [18]:
#merge cluster labels with df_topvenues

df_merged=df_new.copy()
df_merged["Cluster_Labels"]=kmeans.labels_
df_merged=df_merged.join(df_topvenues.drop(["Borough","Neighborhood"],1).set_index("PostalCode"),on="PostalCode")
print(df_merged.shape)
df_merged.sort_values(['Cluster_Labels'],inplace=True)
df_merged

(39, 16)


PostalCode                 Borough  \
1         M4J  East York/East Toronto   
11        M4W        Downtown Toronto   
19        M5H        Downtown Toronto   
28        M5V        Downtown Toronto   
29        M5W  Downtown Toronto Stn A   
30        M5X        Downtown Toronto   
22        M5L        Downtown Toronto   
21        M5K        Downtown Toronto   
20        M5J        Downtown Toronto   
18        M5G        Downtown Toronto   
17        M5E        Downtown Toronto   
16        M5C        Downtown Toronto   
15        M5B        Downtown Toronto   
14        M5A        Downtown Toronto   
13        M4Y        Downtown Toronto   
33        M6J            West Toronto   
34        M6K            West Toronto   
7         M4R         Central Toronto   
35        M6P            West Toronto   
4         M4M            East Toronto   
36        M6R            West Toronto   
2         M4K            East Toronto   
26        M5S        Downtown Toronto   
27        M5T        Downtown Toronto   
9         M4T         Central Toronto   
5         M4N         Central Toronto   
32        M6H            West Toronto   
31        M6G        Downtown Toronto   
0         M4E            East Toronto   
24        M5P         Central Toronto   
37        M6S            West Toronto   
12        M4X        Downtown Toronto   
10        M4V         Central Toronto   
8         M4S         Central Toronto   
6         M4P         Central Toronto   
3         M4L            East Toronto   
25        M5R         Central Toronto   
38        M7Y   East Toronto Business   
23        M5N         Central Toronto   

                                         Neighborhood   Latitude  Longitude  \
1                                  The Danforth  East  43.685347 -79.338106   
11                                           Rosedale  43.679563 -79.377529   
19                           Richmond, Adelaide, King  43.650571 -79.384568   
28  CN Tower, King and Spadina, Railway Lands, Har...  43.628947 -79.394420   
29                                     Enclave of M5E  43.646435 -79.374846   
30             First Canadian Place, Underground city  43.648429 -79.382280   
22                     Commerce Court, Victoria Hotel  43.648198 -79.379817   
21           Toronto Dominion Centre, Design Exchange  43.647177 -79.381576   
20  Harbourfront East, Union Station, Toronto Islands  43.640816 -79.381752   
18                                 Central Bay Street  43.657952 -79.387383   
17                                        Berczy Park  43.644771 -79.373306   
16                                     St. James Town  43.651494 -79.375418   
15                           Garden District, Ryerson  43.657162 -79.378937   
14                          Regent Park, Harbourfront  43.654260 -79.360636   
13                               Church and Wellesley  43.665860 -79.383160   
33                           Little Portugal, Trinity  43.647927 -79.419750   
34       Brockton, Parkdale Village, Exhibition Place  43.636847 -79.428191   
7                                  North Toronto West  43.715383 -79.405678   
35                      High Park, The Junction South  43.661608 -79.464763   
4                                     Studio District  43.659526 -79.340923   
36                             Parkdale, Roncesvalles  43.648960 -79.456325   
2                        The Danforth West, Riverdale  43.679557 -79.352188   
26                     University of Toronto, Harbord  43.662696 -79.400049   
27          Kensington Market, Chinatown, Grange Park  43.653206 -79.400049   
9                         Moore Park, Summerhill East  43.689574 -79.383160   
5                                       Lawrence Park  43.728020 -79.388790   
32                       Dufferin, Dovercourt Village  43.669005 -79.442259   
31                                           Christie  43.669542 -79.422564   
0                                         The Beaches  43.676357 -79.29

In [19]:
#visualizing the clusters using folium and matplotlib colors to indentify differnt clusters 

map_cluster = folium.Map(location=[lat,lon],zoom_start=10)

x=np.arange(k)
y=[i+x+(i*x)**2 for i in range(k)]
colors=cm.rainbow(np.linspace(0,1,len(y)))
rainbow = [color.rgb2hex(i) for i in colors]
markers=[]

for lt,lg,post,b,n,c in zip(df_merged['Latitude'], df_merged['Longitude'], df_merged['PostalCode'], df_merged['Borough'], df_merged['Neighborhood'], df_merged['Cluster_Labels']):
    label = '{} ({}): {} - Cluster {}'.format(b,post,n,c)
    label=folium.Popup(label,parse_html=True)
    folium.CircleMarker([lt,lg], radius=5, popup=label, color=rainbow[c-1], fill=True, fill_color=rainbow[c-1], fill_opacity=0.7).add_to(map_cluster)

map_cluster

In [20]:
#examining the clusters

#CLUSTER 0

df_merged.loc[df_merged['Cluster_Labels']==0, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
1   East York/East Toronto               0                  Park   
11        Downtown Toronto               0                  Park   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
1      Convenience Store              Wine Bar        Discount Store   
11            Playground                 Trail              Wine Bar   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
1     Falafel Restaurant           Event Space  Ethiopian Restaurant   
11          Dessert Shop  Ethiopian Restaurant           Escape Room   

   8th Most Common Venue        9th Most Common Venue  \
1            Escape Room            Electronics Store   
11     Electronics Store  Eastern European Restaurant   

         10th Most Common Venue  
1   Eastern European Restaurant  
11          Dumpling Restaurant

In [21]:
#CLUSTER 1

df_merged.loc[df_merged['Cluster_Labels']==1, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
19        Downtown Toronto               1           Coffee Shop   
28        Downtown Toronto               1      Airport Terminal   
29  Downtown Toronto Stn A               1           Coffee Shop   
30        Downtown Toronto               1           Coffee Shop   
22        Downtown Toronto               1           Coffee Shop   
21        Downtown Toronto               1           Coffee Shop   
20        Downtown Toronto               1           Coffee Shop   
18        Downtown Toronto               1           Coffee Shop   
17        Downtown Toronto               1           Coffee Shop   
16        Downtown Toronto               1           Coffee Shop   
15        Downtown Toronto               1           Coffee Shop   
14        Downtown Toronto               1           Coffee Shop   
13        Downtown Toronto               1           Coffee Shop   
33            West Toronto               1                   Bar   
34            West Toronto               1                  Café   
7          Central Toronto               1           Coffee Shop   
35            West Toronto               1    Mexican Restaurant   
4             East Toronto               1           Coffee Shop   
36            West Toronto               1             Gift Shop   
2             East Toronto               1      Greek Restaurant   
26        Downtown Toronto               1                  Café   
27        Downtown Toronto               1                  Café   

   2nd Most Common Venue          3rd Most Common Venue  \
19                  Café                     Restaurant   
28       Airport Service               Sculpture Garden   
29          Cocktail Bar             Seafood Restaurant   
30                  Café                          Hotel   
22            Restaurant                           Café   
21                 Hotel                           Café   
20              Aquarium                           Café   
18    Italian Restaurant                           Café   
17          Cocktail Bar                         Bakery   
16                  Café                      Gastropub   
15        Clothing Store             Italian Restaurant   
14                Bakery                           Park   
13   Japanese Restaurant               Sushi Restaurant   
33           Men's Store                    Coffee Shop   
34        Breakfast Spot                      Nightclub   
7         Clothing Store                    Yoga Studio   
35       Thai Restaurant                           Café   
4                   Café                         Bakery   
36        Breakfast Spot    Eastern European Restaurant   
2            Coffee Shop             Italian Restaurant   
26                   Bar            Japanese Restaurant   
27           Coffee Shop  Vegetarian / Vegan Restaurant   

            4th Most Common Venue   5th Most Common Venue  \
19                Thai Restaurant                     Gym   
28                          Plane           Boat or Ferry   
29                     Restaurant                  Bakery   
30                            Gym     Japanese Restaurant   
22                          Hotel      Italian Restaurant   
21                     Restaurant             Salad Place   
20                          Hotel      Italian Restaurant   
18                 Sandwich Place     Japanese Restaurant   
17                       Pharmacy             Cheese Shop   
16                   Cocktail Bar                     Gym   
15      Middle Eastern Restaurant          Cosmetics Shop   
14                 Breakfast Spot                    Café   
13                     Restaurant                 Gay Bar   
33  Vegetarian / Vegan Restaurant        Asian Restaurant   
34                         Bakery             Coffee Shop   
7              Salon / Barbershop              Restaurant   
35              Convenience Store                     Bar  

In [22]:
#CLUSTER 2

df_merged.loc[df_merged['Cluster_Labels']==2, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
9  Central Toronto               2            Restaurant   

  2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
9                  Park                 Trail           Summer Camp   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
9  Ethiopian Restaurant           Escape Room     Electronics Store   

         8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
9  Eastern European Restaurant   Dumpling Restaurant             Donut Shop

In [23]:
#CLUSTER 3

df_merged.loc[df_merged['Cluster_Labels']==3, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
5  Central Toronto               3           Swim School   

  2nd Most Common Venue       3rd Most Common Venue 4th Most Common Venue  \
5                  Park  Construction & Landscaping              Bus Line   

  5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
5              Wine Bar           Event Space  Ethiopian Restaurant   

  8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
5           Escape Room     Electronics Store  Eastern European Restaurant

In [24]:
#CLUSTER 4

df_merged.loc[df_merged['Cluster_Labels']==4, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
32           West Toronto               4              Pharmacy   
31       Downtown Toronto               4         Grocery Store   
0            East Toronto               4                 Trail   
24        Central Toronto               4                 Trail   
37           West Toronto               4                  Café   
12       Downtown Toronto               4            Restaurant   
10        Central Toronto               4           Coffee Shop   
8         Central Toronto               4           Pizza Place   
6         Central Toronto               4      Department Store   
3            East Toronto               4  Fast Food Restaurant   
25        Central Toronto               4        Sandwich Place   
38  East Toronto Business               4    Light Rail Station   

   2nd Most Common Venue      3rd Most Common Venue 4th Most Common Venue  \
32                Bakery  Middle Eastern Restaurant                  Park   
31                  Café                       Park             Nightclub   
0                    Pub          Health Food Store              Wine Bar   
24         Jewelry Store         Mexican Restaurant      Sushi Restaurant   
37           Coffee Shop                        Pub           Pizza Place   
12                  Park                Coffee Shop                  Café   
10      Sushi Restaurant        American Restaurant                  Bank   
8           Dessert Shop             Sandwich Place                   Gym   
6                   Park                Pizza Place  Gym / Fitness Center   
3                    Gym          Food & Drink Shop          Liquor Store   
25                  Café                Coffee Shop             BBQ Joint   
38            Comic Shop             Farmers Market            Restaurant   

        5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
32              Grocery Store               Brewery           Supermarket   
31                 Restaurant                  Bank    Italian Restaurant   
0                  Donut Shop        Discount Store   Distribution Center   
24                   Wine Bar                 Diner  Ethiopian Restaurant   
37         Italian Restaurant      Sushi Restaurant                   Gym   
12         Italian Restaurant           Pizza Place                Bakery   
10                        Pub           Supermarket    Light Rail Station   
8                        Café    Italian Restaurant      Sushi Restaurant   
6              Breakfast Spot            Playground     Food & Drink Shop   
3              Sandwich Place    Italian Restaurant            Restaurant   
25  Middle Eastern Restaurant           Flower Shop                  Park   
38              Auto Workshop         Burrito Place  Fast Food Restaurant   

   8th Most Common Venue 9th Most Common Venue       10th Most Common Venue  
32          Liquor Store  Brazilian Restaurant                  Music Venue  
31            Baby Store           Coffee Shop                  Candy Store  
0                Dog Run      Doner Restaurant  Eastern European Restaurant  
24           Escape Room     Electronics Store  Eastern European Restaurant  
37     French Restaurant            Restaurant                         Park  
12                   Pub        Sandwich Place            Convenience Store  
10   Fried Chicken Joint            Restaurant                   Bagel Shop  
8            Coffee Shop                 Diner               Farmers Market  
6                  Hotel        Sandwich Place                      Dog Run  
3      Fish & Chips Shop        Ice Cream Shop                   Steakhouse  
25          Burger Joint            Donut Shop                     Pharmacy  
38                   Spa      Recording Studio                Garden Center

In [25]:
#CLUSTER 5

df_merged.loc[df_merged['Cluster_Labels']==5, df_merged.columns[[1] + list(range(5,df_merged.shape[1]))]]

Borough  Cluster_Labels 1st Most Common Venue  \
23  Central Toronto               5          Home Service   

   2nd Most Common Venue 3rd Most Common Venue 4th Most Common Venue  \
23           Music Venue                Garden                  Pool   

   5th Most Common Venue 6th Most Common Venue 7th Most Common Venue  \
23              Wine Bar        Discount Store   Distribution Center   

   8th Most Common Venue 9th Most Common Venue 10th Most Common Venue  
23               Dog Run      Doner Restaurant             Donut Shop

###  Cluster 0 contains park, playground, and wine bar

### Cluster 1 is a very ***popular*** place because it has coffee shops, restaurants, bars, hotels, airport too

### Cluster 2 contains restaurant and a park

### Cluster 3 contain swim school and park

### Cluster 4 is quite ***popular*** too because it has pharmacy, grocery and departmental stores

### Cluster 5 contains garden, pool and music venue